In [ ]:
import sys
from boto3 import Session as BotoSession
import sagemaker
from sagemaker import Session
from sagemaker.pytorch import PyTorchModel, PyTorchPredictor

In [ ]:
print('Python version: {}'. format(sys.version))
print('Sagemaker version: {}'.format(sagemaker.__version__))

In [ ]:
boto_session=BotoSession(
    region_name='eu-west-1',
    profile_name='defeatcovid19'
)
session = Session(boto_session=boto_session)

In [ ]:
version=5

In [ ]:
role = 'arn:aws:iam::953018779520:role/service-role/AmazonSageMaker-ExecutionRole-20200327T152761'
pytorch_model = PyTorchModel(
    model_data='s3://prod.models.sagemaker.defeatcovid19.org/pilot-akinesis-eco-sacco/model.tar.gz', 
    role=role, 
    entry_point='inference.py',
    framework_version='1.6.0',
    py_version='py3',
    sagemaker_session=session,
    name='PilotEcoAkinesisSacco-v{}'.format(version)
)
predictor = pytorch_model.deploy(
    instance_type='ml.t2.medium', 
    initial_instance_count=1,
    endpoint_name='PilotEcoAkinesisSaccoEndpoint-v{}'.format(version)
)

In [ ]:
old_predictor = PyTorchPredictor(
    endpoint_name='PilotEcoAkinesisSaccoEndpoint-v{}'.format(version - 1),
    sagemaker_session=session
)
old_predictor.delete_endpoint()
old_predictor.delete_model()